In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 46.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=792d29283137e7027ec32d7a3217e57756631cdf6cf27b06452f782cda1415b7
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import requests
import numpy as np
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
spark = SparkSession\
          .builder \
          .master("local[*]") \
          .appName('nba') \
          .getOrCreate()

In [ ]:
games_df = spark.read\
              .option('header', 'true')\
              .csv("/content/drive/MyDrive/NBA_Scrape/data/games/*", inferSchema= True)

In [ ]:
games_df.count(), len(games_df.columns)

(32297, 11)

In [ ]:
games_df.show(5)

+---------------+--------------------+----------+--------------------+----------+----------+-----+-------+-------------------+---------+--------------------+
|seasonStartYear|            awayTeam|pointsAway|            homeTeam|pointsHome|attendance|notes|startET|           datetime|isRegular|             game_id|
+---------------+--------------------+----------+--------------------+----------+----------+-----+-------+-------------------+---------+--------------------+
|           2018|  Philadelphia 76ers|      87.0|      Boston Celtics|     105.0|   18624.0| null|  8:00p|2018-10-16 00:00:00|        1|386daeeab4034a75b...|
|           2018|Oklahoma City Thu...|     100.0|Golden State Warr...|     108.0|   19596.0| null| 10:30p|2018-10-16 00:00:00|        1|1d3f8cfeb7134600a...|
|           2018|New Orleans Pelicans|     131.0|     Houston Rockets|     112.0|   18055.0| null|  8:00p|2018-10-17 00:00:00|        1|69fbb9cc8ab1478ba...|
|           2018|     Milwaukee Bucks|     113.0|   

In [ ]:
games_df.printSchema()

root
 |-- seasonStartYear: integer (nullable = true)
 |-- awayTeam: string (nullable = true)
 |-- pointsAway: double (nullable = true)
 |-- homeTeam: string (nullable = true)
 |-- pointsHome: double (nullable = true)
 |-- attendance: double (nullable = true)
 |-- notes: string (nullable = true)
 |-- startET: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- isRegular: integer (nullable = true)
 |-- game_id: string (nullable = true)



In [ ]:
games_df = games_df\
            .withColumn("pointsAway", games_df.pointsAway.cast("integer"))\
            .withColumn("pointsHome", games_df.pointsHome.cast("integer"))\
            .withColumn("attendance", games_df.attendance.cast("integer"))\
            .withColumn("datetime", games_df.datetime.cast(types.DateType()))

In [ ]:
games_df.printSchema()

root
 |-- seasonStartYear: integer (nullable = true)
 |-- awayTeam: string (nullable = true)
 |-- pointsAway: integer (nullable = true)
 |-- homeTeam: string (nullable = true)
 |-- pointsHome: integer (nullable = true)
 |-- attendance: integer (nullable = true)
 |-- notes: string (nullable = true)
 |-- startET: string (nullable = true)
 |-- datetime: date (nullable = true)
 |-- isRegular: integer (nullable = true)
 |-- game_id: string (nullable = true)



In [ ]:
games_df.show(5)

+---------------+--------------------+----------+--------------------+----------+----------+-----+-------+----------+---------+--------------------+
|seasonStartYear|            awayTeam|pointsAway|            homeTeam|pointsHome|attendance|notes|startET|  datetime|isRegular|             game_id|
+---------------+--------------------+----------+--------------------+----------+----------+-----+-------+----------+---------+--------------------+
|           2018|  Philadelphia 76ers|        87|      Boston Celtics|       105|     18624| null|  8:00p|2018-10-16|        1|386daeeab4034a75b...|
|           2018|Oklahoma City Thu...|       100|Golden State Warr...|       108|     19596| null| 10:30p|2018-10-16|        1|1d3f8cfeb7134600a...|
|           2018|New Orleans Pelicans|       131|     Houston Rockets|       112|     18055| null|  8:00p|2018-10-17|        1|69fbb9cc8ab1478ba...|
|           2018|     Milwaukee Bucks|       113|   Charlotte Hornets|       112|     17889| null|  7:00p|

In [ ]:
salary_df = spark.read\
              .option('header', 'true')\
              .csv("/content/drive/MyDrive/NBA_Scrape/data/playersalary/*", inferSchema= True)

In [ ]:
salary_df.count()

12815

In [ ]:
salary_df.show(5)

+----------------+---------------+-----------+------------------+
|      playerName|seasonStartYear|     salary|inflationAdjSalary|
+----------------+---------------+-----------+------------------+
|  Michael Jordan|           1996|$30,140,000|       $52,258,566|
|    Horace Grant|           1996|$14,857,000|       $25,759,971|
|   Reggie Miller|           1996|$11,250,000|       $19,505,934|
|Shaquille O'Neal|           1996|$10,714,000|       $18,576,585|
|     Gary Payton|           1996|$10,212,000|       $17,706,187|
+----------------+---------------+-----------+------------------+
only showing top 5 rows



In [ ]:
salary_df = salary_df.coalesce(1)\
                    .dropDuplicates(subset=['playerName', 'seasonStartYear', "salary"])

In [ ]:
players_df = spark.read\
                .option("header", "true")\
                .csv("/content/drive/MyDrive/NBA_Scrape/data/playerinfo/*", inferSchema=True)

In [ ]:
players_df.count(), len(players_df.columns)

(5023, 9)

In [ ]:
players_df.show(5)

+--------------------+----+----+---+----+-----+----------------+--------------------+--------------------+
|          playerName|From|  To|Pos|  Ht|   Wt|       birthDate|            Colleges|             img_url|
+--------------------+----+----+---+----+-----+----------------+--------------------+--------------------+
|      Alaa Abdelnaby|1991|1995|F-C|6-10|240.0|   June 24, 1968|                Duke|https://www.baske...|
|     Zaid Abdul-Aziz|1969|1978|C-F| 6-9|235.0|   April 7, 1946|          Iowa State|                null|
|Kareem Abdul-Jabbar*|1970|1989|  C| 7-2|225.0|  April 16, 1947|                UCLA|https://www.baske...|
|  Mahmoud Abdul-Rauf|1991|2001|  G| 6-1|162.0|   March 9, 1969|                 LSU|                null|
|   Tariq Abdul-Wahad|1998|2003|  F| 6-6|223.0|November 3, 1974|Michigan, San Jos...|                null|
+--------------------+----+----+---+----+-----+----------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
players_df.printSchema()

root
 |-- playerName: string (nullable = true)
 |-- From: integer (nullable = true)
 |-- To: integer (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Ht: string (nullable = true)
 |-- Wt: double (nullable = true)
 |-- birthDate: string (nullable = true)
 |-- Colleges: string (nullable = true)
 |-- img_url: string (nullable = true)



In [ ]:
# players_df.select([F.count(F.when(
#                            F.isnan(c) |
#                            F.col(c).isNull() | 
#                            (F.col(c) == "") |
#                            F.col(c).contains('None') |
#                            F.col(c).contains('Null'), c
#                            )
#                        ).alias(c) for c in players_df.columns]
#    ).show()

In [ ]:
players_df.select('playerName').distinct().count()

4980

In [ ]:
player_dup = list(players_df.groupBy('playerName').count().filter("count > 1").select('playerName').toPandas()['playerName'].values)
players_df.filter(players_df.playerName.isin(player_dup)).show(44)

+----------------+----+----+---+----+-----+------------------+--------------------+--------------------+
|      playerName|From|  To|Pos|  Ht|   Wt|         birthDate|            Colleges|             img_url|
+----------------+----+----+---+----+-----+------------------+--------------------+--------------------+
|    Dan Anderson|1968|1969|  C|6-10|230.0| February 15, 1943|    Augsburg College|                null|
|    Dan Anderson|1975|1976|  G| 6-2|185.0|   January 1, 1951|                 USC|https://www.baske...|
|       Dee Brown|1991|2002|  G| 6-1|160.0| November 29, 1968|Jacksonville Univ...|                null|
|       Dee Brown|2007|2009|  G| 6-0|185.0|   August 17, 1984|            Illinois|                null|
|      Mark Davis|1989|1989|F-G| 6-6|195.0|      June 8, 1963|        Old Dominion|                null|
|      Mark Davis|1996|2000|G-F| 6-7|210.0|    April 26, 1973|Howard College, T...|                null|
|      Mike Davis|1970|1973|  G| 6-3|185.0|     July 26

In [ ]:
def createExp(end, start):
  if start == None:
    return None
  else:
    return end-start 

yr_diff_udf = F.udf(createExp, returnType=types.IntegerType())

In [ ]:
players_df = players_df\
                .withColumn("hall_of_fame", F.regexp_replace(F.regexp_extract(F.col('playerName'), "\*", 0), "\*", "Yes"))\
                .withColumn("playerName", F.regexp_replace(F.col('playerName'), "\*", "").cast(types.StringType()))\
                .withColumn("birthDate", F.to_date(players_df.birthDate, "MMMM d, yyyy").cast(types.DateType()))\
                .withColumnRenamed("From", "start_career")\
                .withColumnRenamed("To", "end_career")\
                .withColumnRenamed("Pos", "position")\
                .withColumnRenamed("Ht", "height")\
                .withColumnRenamed('Wt', 'weight')\
                .withColumnRenamed("birthDate", "birth_date")\
                .withColumnRenamed('playerName', 'player_name')

In [ ]:
players_df.show(10)

+-------------------+------------+----------+--------+------+------+----------+--------------------+--------------------+------------+
|        player_name|start_career|end_career|position|height|weight|birth_date|            Colleges|             img_url|hall_of_fame|
+-------------------+------------+----------+--------+------+------+----------+--------------------+--------------------+------------+
|     Alaa Abdelnaby|        1991|      1995|     F-C|  6-10| 240.0|1968-06-24|                Duke|https://www.baske...|            |
|    Zaid Abdul-Aziz|        1969|      1978|     C-F|   6-9| 235.0|1946-04-07|          Iowa State|                null|            |
|Kareem Abdul-Jabbar|        1970|      1989|       C|   7-2| 225.0|1947-04-16|                UCLA|https://www.baske...|         Yes|
| Mahmoud Abdul-Rauf|        1991|      2001|       G|   6-1| 162.0|1969-03-09|                 LSU|                null|            |
|  Tariq Abdul-Wahad|        1998|      2003|       F| 

In [ ]:
players_df = players_df\
                .withColumn("experience", F.col('end_career')-F.col('start_career'))\
                .withColumn('career_start_age', yr_diff_udf(F.col('start_career'), F.year(F.col('birth_date'))))\
                .withColumn('career_end_age', yr_diff_udf(F.col('end_career'), F.year(F.col('birth_date'))))\
                .withColumn('img_url', F.trim(F.col('img_url')))

In [ ]:
players_df.show()

+-------------------+------------+----------+--------+------+------+----------+--------------------+--------------------+------------+----------+----------------+--------------+
|        player_name|start_career|end_career|position|height|weight|birth_date|            Colleges|             img_url|hall_of_fame|experience|career_start_age|career_end_age|
+-------------------+------------+----------+--------+------+------+----------+--------------------+--------------------+------------+----------+----------------+--------------+
|     Alaa Abdelnaby|        1991|      1995|     F-C|  6-10| 240.0|1968-06-24|                Duke|https://www.baske...|            |         4|              23|            27|
|    Zaid Abdul-Aziz|        1969|      1978|     C-F|   6-9| 235.0|1946-04-07|          Iowa State|                null|            |         9|              23|            32|
|Kareem Abdul-Jabbar|        1970|      1989|       C|   7-2| 225.0|1947-04-16|                UCLA|https://ww

In [ ]:
player_dup = list(players_df.groupBy('player_name').count().filter("count > 1").select('player_name').toPandas()['player_name'].values)
players_df.filter(players_df.player_name.isin(player_dup)).count()

91

In [ ]:
# players_df = players_df\
#                   .sort(F.col('player_name').asc(), 
#                         F.col('experience').desc(), 
#                         F.col('start_career').desc(), 
#                         F.col('end_career').desc())\
#                   .coalesce(1)\
#                   .dropDuplicates(subset=['player_name'])

In [ ]:
players_df.show()

+-------------------+------------+----------+--------+------+------+----------+--------------------+--------------------+------------+----------+----------------+--------------+
|        player_name|start_career|end_career|position|height|weight|birth_date|            Colleges|             img_url|hall_of_fame|experience|career_start_age|career_end_age|
+-------------------+------------+----------+--------+------+------+----------+--------------------+--------------------+------------+----------+----------------+--------------+
|     Alaa Abdelnaby|        1991|      1995|     F-C|  6-10| 240.0|1968-06-24|                Duke|https://www.baske...|            |         4|              23|            27|
|    Zaid Abdul-Aziz|        1969|      1978|     C-F|   6-9| 235.0|1946-04-07|          Iowa State|                null|            |         9|              23|            32|
|Kareem Abdul-Jabbar|        1970|      1989|       C|   7-2| 225.0|1947-04-16|                UCLA|https://ww

In [ ]:
players_df.count(), len(players_df.columns)

(5023, 13)

In [ ]:
print(len(player_dup))
players_df.filter(players_df.player_name.isin(player_dup)).sort('player_name').show(100)

44
+----------------+------------+----------+--------+------+------+----------+--------------------+--------------------+------------+----------+----------------+--------------+
|     player_name|start_career|end_career|position|height|weight|birth_date|            Colleges|             img_url|hall_of_fame|experience|career_start_age|career_end_age|
+----------------+------------+----------+--------+------+------+----------+--------------------+--------------------+------------+----------+----------------+--------------+
|    Bill Bradley|        1968|      1968|       G|  5-11| 165.0|1941-06-16|                null|https://www.baske...|            |         0|              27|            27|
|    Bill Bradley|        1968|      1977|     F-G|   6-5| 205.0|1943-07-28|           Princeton|                null|         Yes|         9|              25|            34|
|       Bob Duffy|        1947|      1947|       F|   6-4| 175.0|1922-07-05|              Tulane|                null|    

In [ ]:
scores_df = spark.read\
                .option("header", "true")\
                .csv("/content/drive/MyDrive/NBA_Scrape/data/boxscores/*", inferSchema=True)

In [ ]:
scores_df.count(), len(scores_df.columns)

(791476, 21)

In [ ]:
scores_df.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- teamName: string (nullable = true)
 |-- playerName: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- FG: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- 3P: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- FT: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- ORB: string (nullable = true)
 |-- DRB: string (nullable = true)
 |-- TRB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- +/-: string (nullable = true)
 |-- isStarter: integer (nullable = true)



In [ ]:
scores_df.show(5)

+--------------------+-------------+----------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+
|             game_id|     teamName|      playerName|   MP| FG|FGA| 3P|3PA| FT|FTA|ORB|DRB|TRB|AST|STL|BLK|TOV| PF|PTS|+/-|isStarter|
+--------------------+-------------+----------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+
|cbfc245675834b09a...|Orlando Magic|   Arron Afflalo|32:59|  3| 14|  1|  5|  2|  3|  1|  2|  3|  1|  0|  0|  1|  1|  9|-14|        1|
|cbfc245675834b09a...|Orlando Magic|  Nikola Vučević|30:39|  4| 11|  0|  0|  0|  0|  5|  5| 10|  3|  2|  1|  5|  4|  8|-22|        1|
|cbfc245675834b09a...|Orlando Magic|   Jameer Nelson|30:34|  4| 13|  3|  7|  1|  1|  1|  4|  5|  7|  2|  0|  2|  1| 12|-11|        1|
|cbfc245675834b09a...|Orlando Magic|   Jason Maxiell|26:19|  0|  5|  0|  0|  0|  0|  1|  4|  5|  0|  1|  2|  2|  2|  0|-15|        1|
|cbfc245675834b09a...|Orlando Magic|Maurice Harkless|23:30|  6

#### Filter out values

In [ ]:
filter_values = [
        "Did Not Dress", 
        "Player Suspended",
        "Did Not Play", 
        "Not With Team"
    ]

#### Convert columns datatypes

In [ ]:
import regex
import text_unidecode
import re
import unicodedata

In [ ]:
def remove_accent_chars_regex(x: str):
    translationTable = str.maketrans("ğĞıİöÖüÜşŞçÇß", "gGiIoOuUsScCs")
    x = re.sub(r'[^\w\s]', '', x.translate(translationTable))
    return regex.sub(r'\p{Mn}', '', unicodedata.normalize('NFKD', x))

player_name_udf = F.udf(remove_accent_chars_regex, returnType=types.StringType())

In [ ]:
scores_df = scores_df\
              .filter(~scores_df.MP.isin(filter_values))\
              .withColumn('FG', F.col('FG').cast(types.IntegerType()))\
              .withColumn('FGA', F.col('FGA').cast(types.IntegerType()))\
              .withColumn('3P', F.col('3P').cast(types.IntegerType()))\
              .withColumn('3PA', F.col('3PA').cast(types.IntegerType()))\
              .withColumn('FT', F.col('FT').cast(types.IntegerType()))\
              .withColumn('FTA', F.col('FTA').cast(types.IntegerType()))\
              .withColumn('ORB', F.col('ORB').cast(types.IntegerType()))\
              .withColumn('DRB', F.col('DRB').cast(types.IntegerType()))\
              .withColumn('TRB', F.col('TRB').cast(types.IntegerType()))\
              .withColumn('AST', F.col('AST').cast(types.IntegerType()))\
              .withColumn('STL', F.col('STL').cast(types.IntegerType()))\
              .withColumn('BLK', F.col('BLK').cast(types.IntegerType()))\
              .withColumn('TOV', F.col('TOV').cast(types.IntegerType()))\
              .withColumn('PF', F.col('PF').cast(types.IntegerType()))\
              .withColumn('PTS', F.col('PTS').cast(types.IntegerType()))\
              .withColumn('+/-', F.col('+/-').cast(types.IntegerType()))\
              .withColumn('minutes_played', F.round(F.split(F.col('MP'), ":")[0]+F.split(F.col('MP'), ":")[1]/60, 2))\
              .withColumn('playerName', player_name_udf(F.col('playerName')))\
              .withColumnRenamed('teamName', 'team_name')\
              .withColumnRenamed('playerName', 'player_name')\
              .withColumnRenamed('+/-', 'plus_minus')\
              .select(['game_id', 'team_name', 'player_name', 'minutes_played', 'FG', 'FGA', '3P',
                       '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
                        'TOV', 'PF', 'PTS', 'plus_minus', 'isStarter'])

In [ ]:
scores_df.show(5)

+--------------------+-------------+----------------+--------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----------+---------+
|             game_id|    team_name|     player_name|minutes_played| FG|FGA| 3P|3PA| FT|FTA|ORB|DRB|TRB|AST|STL|BLK|TOV| PF|PTS|plus_minus|isStarter|
+--------------------+-------------+----------------+--------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----------+---------+
|cbfc245675834b09a...|Orlando Magic|   Arron Afflalo|         32.98|  3| 14|  1|  5|  2|  3|  1|  2|  3|  1|  0|  0|  1|  1|  9|       -14|        1|
|cbfc245675834b09a...|Orlando Magic|  Nikola Vucevic|         30.65|  4| 11|  0|  0|  0|  0|  5|  5| 10|  3|  2|  1|  5|  4|  8|       -22|        1|
|cbfc245675834b09a...|Orlando Magic|   Jameer Nelson|         30.57|  4| 13|  3|  7|  1|  1|  1|  4|  5|  7|  2|  0|  2|  1| 12|       -11|        1|
|cbfc245675834b09a...|Orlando Magic|   Jason Maxiell|         26.32|  0|  5|  0|  0|  0|  0|  1|  4|

In [ ]:
scores_df.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- team_name: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- minutes_played: double (nullable = true)
 |-- FG: integer (nullable = true)
 |-- FGA: integer (nullable = true)
 |-- 3P: integer (nullable = true)
 |-- 3PA: integer (nullable = true)
 |-- FT: integer (nullable = true)
 |-- FTA: integer (nullable = true)
 |-- ORB: integer (nullable = true)
 |-- DRB: integer (nullable = true)
 |-- TRB: integer (nullable = true)
 |-- AST: integer (nullable = true)
 |-- STL: integer (nullable = true)
 |-- BLK: integer (nullable = true)
 |-- TOV: integer (nullable = true)
 |-- PF: integer (nullable = true)
 |-- PTS: integer (nullable = true)
 |-- plus_minus: integer (nullable = true)
 |-- isStarter: integer (nullable = true)



In [ ]:
players_df.createOrReplaceTempView('players')
salary_df.createOrReplaceTempView('salary')

In [ ]:
scores_df.createOrReplaceTempView('scores')
games_df.createOrReplaceTempView('games')

In [ ]:
players_df.filter(F.col('player_name') == "Aaron Brooks").show()

+------------+------------+----------+--------+------+------+----------+--------+--------------------+------------+----------+----------------+--------------+
| player_name|start_career|end_career|position|height|weight|birth_date|Colleges|             img_url|hall_of_fame|experience|career_start_age|career_end_age|
+------------+------------+----------+--------+------+------+----------+--------+--------------------+------------+----------+----------------+--------------+
|Aaron Brooks|        2008|      2018|       G|   6-0| 161.0|1985-01-14|  Oregon|https://www.baske...|            |        10|              23|            33|
+------------+------------+----------+--------+------+------+----------+--------+--------------------+------------+----------+----------------+--------------+



In [ ]:
scores_df.count()

661825

In [ ]:
spark.sql(
    """
    SELECT *
      FROM games
      JOIN scores
      USING(game_id)
    """
).show(3)

+--------------------+---------------+-------------+----------+--------------+----------+----------+-----+-------+----------+---------+-------------+--------------+--------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----------+---------+
|             game_id|seasonStartYear|     awayTeam|pointsAway|      homeTeam|pointsHome|attendance|notes|startET|  datetime|isRegular|    team_name|   player_name|minutes_played| FG|FGA| 3P|3PA| FT|FTA|ORB|DRB|TRB|AST|STL|BLK|TOV| PF|PTS|plus_minus|isStarter|
+--------------------+---------------+-------------+----------+--------------+----------+----------+-----+-------+----------+---------+-------------+--------------+--------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----------+---------+
|cbfc245675834b09a...|           2013|Orlando Magic|        87|Indiana Pacers|        97|     18165| null|   null|2013-10-29|        1|Orlando Magic| Arron Afflalo|         32.98|  3| 14|  1|  5|  2|  3|  1|  2|  3|  

In [ ]:
spark.sql(
    """
    select * from players
    """
).show(3)

+-------------------+------------+----------+--------+------+------+----------+----------+--------------------+------------+----------+----------------+--------------+
|        player_name|start_career|end_career|position|height|weight|birth_date|  Colleges|             img_url|hall_of_fame|experience|career_start_age|career_end_age|
+-------------------+------------+----------+--------+------+------+----------+----------+--------------------+------------+----------+----------------+--------------+
|     Alaa Abdelnaby|        1991|      1995|     F-C|  6-10| 240.0|1968-06-24|      Duke|https://www.baske...|            |         4|              23|            27|
|    Zaid Abdul-Aziz|        1969|      1978|     C-F|   6-9| 235.0|1946-04-07|Iowa State|                null|            |         9|              23|            32|
|Kareem Abdul-Jabbar|        1970|      1989|       C|   7-2| 225.0|1947-04-16|      UCLA|https://www.baske...|         Yes|        19|              23|        

In [ ]:
spark.sql(
    """
    SELECT 
        *
    FROM (
      SELECT *
      FROM games
      JOIN scores
      USING(game_id)
      ) g
    JOIN players p
    ON g.player_name = p.player_name
      AND (year(g.datetime) >= p.start_career AND year(g.datetime) <= p.end_career)
    """
).count()

578628

In [ ]:
game_stat_yr = \
spark.sql("""
WITH join_scores AS (
  SELECT 
        g.* -- , s.salary
  FROM (
    SELECT *
    FROM games
    JOIN scores
    USING(game_id)
    ) g/*
  LEFT JOIN salary s
  ON g.player_name = s.playerName
    AND g.seasonStartYear = s.seasonStartYear*/
)
SELECT 
      concat(gs.seasonStartYear, "-", RIGHT(FLOOR(gs.seasonStartYear+1), 2)) AS season,
      -- year(gs.datetime) as game_year,
      gs.player_name,
      gs.team_name,
      -- year(gs.datetime) - year(p.birth_date) AS age,
      p.birth_date,
      -- gs.salary,
      -- gs.isRegular,
      -- gs.isStarter,
      count(1) as game_played,
      SUM(isStarter) AS game_started,
      ROUND(SUM(gs.minutes_played)) AS minutes_played,
      SUM(FG) AS FG, SUM(FGA) AS FGA,
      SUM(3P) AS 3P, SUM(3PA) AS 3PA,
      SUM(FT) AS FT, SUM(FTA) AS FTA,
      SUM(ORB) AS ORB, SUM(DRB) AS DRB,
      SUM(TRB) AS TRB, SUM(AST) AS AST,
      SUM(STL) AS STL, SUM(BLK) AS BLK,
      SUM(TOV) AS TOV, SUM(PF) AS PF,
      SUM(PTS) AS PTS, AVG(plus_minus) AS plus_minus
FROM (join_scores) gs
JOIN players p
  ON gs.player_name=p.player_name
  AND (year(gs.datetime) >= p.start_career AND year(gs.datetime) <= p.end_career)
GROUP BY
      1, 2, 3, 4 -- , 5 -- , 6 -- , 7
ORDER BY
      2, 1
""")

In [ ]:
game_stat_yr.filter(game_stat_yr.player_name.isin(player_dup)).show(100)

+-------+----------------+--------------------+----------+-----------+------------+--------------+---+----+---+---+---+---+---+---+---+---+---+---+---+---+----+--------------------+
| season|     player_name|           team_name|birth_date|game_played|game_started|minutes_played| FG| FGA| 3P|3PA| FT|FTA|ORB|DRB|TRB|AST|STL|BLK|TOV| PF| PTS|          plus_minus|
+-------+----------------+--------------------+----------+-----------+------------+--------------+---+----+---+---+---+---+---+---+---+---+---+---+---+---+----+--------------------+
|2006-07|     Bobby Jones|  Philadelphia 76ers|1984-01-09|         31|           5|         261.0| 37|  81|  1|  8| 18| 29| 18| 26| 44| 13| 10|  0| 12| 41|  93|  1.3548387096774193|
|2007-08|     Bobby Jones|     Houston Rockets|1984-01-09|          4|           0|           9.0|  2|   4|  0|  0|  0|  0|  0|  1|  1|  0|  1|  0|  1|  3|   4|                -1.0|
|2007-08|     Bobby Jones|      Denver Nuggets|1984-01-09|         25|           0|       

In [ ]:
game_stat_yr\
        .coalesce(1)\
        .write\
        .option("header", True)\
        .csv("/content/drive/MyDrive/NBA_Scrape/data/season_stats1.csv", mode='overwrite')

In [ ]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [ ]:
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1-3KFXOe9LkIZcYqLjqoHMwMvkjy4-T0-aoVDzLTzEIo").worksheet('season_stats')
set_with_dataframe(sheet, game_stat_yr.toPandas())#.to_csv('/content/drive/MyDrive/NBA_Scrape/data/season_stats.csv', index=False)

In [ ]:
!rm -rf /content/drive/MyDrive/NBA_Scrape/data/season_stats.csv

In [ ]:
games_df.toPandas().to_csv("/content/drive/MyDrive/NBA_Scrape/data/games.csv", index=False)

In [ ]:
players_df.toPandas().to_csv("/content/drive/MyDrive/NBA_Scrape/data/players.csv", index=False)

In [ ]:
scores_df.toPandas().to_csv('/content/drive/MyDrive/NBA_Scrape/data/boxscores.csv', index=False)

In [ ]:
game_stat_yr.printSchema()

root
 |-- season: string (nullable = true)
 |-- game_year: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- team_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- isRegular: integer (nullable = true)
 |-- isStarter: integer (nullable = true)
 |-- game_played: long (nullable = false)
 |-- game_started: long (nullable = true)
 |-- minutes_played: double (nullable = true)
 |-- FG: long (nullable = true)
 |-- FGA: long (nullable = true)
 |-- 3P: long (nullable = true)
 |-- 3PA: long (nullable = true)
 |-- FT: long (nullable = true)
 |-- FTA: long (nullable = true)
 |-- ORB: long (nullable = true)
 |-- DRB: long (nullable = true)
 |-- TRB: long (nullable = true)
 |-- AST: long (nullable = true)
 |-- STL: long (nullable = true)
 |-- BLK: long (nullable = true)
 |-- TOV: long (nullable = true)
 |-- PF: long (nullable = true)
 |-- PTS: long (nullable = true)
 |-- plus_minus: double (nullable = true)

